# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Import cities file into the cities_pd DataFrame
cities_pd = pd.read_csv("./Resources/cities.csv")
cities_pd.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,klaksvik,91.0,FO,1.603841e+09,87.0,62.23,-6.59,279.15,4.60
1,1,hermanus,100.0,ZA,1.603841e+09,93.0,-34.42,19.23,289.26,1.85
2,2,vaitupu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,belushya guba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,attawapiskat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
#check the data 
cities_pd.shape

(584, 10)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps
gmaps.configure(api_key=g_key)
fig = gmaps.figure()

In [5]:
#Drop na 
citiesna=cities_pd.dropna()

In [6]:
# Store latitude and longitude in locations
locations =citiesna[["Lat", "Lng"]]
#locations.shape

In [7]:
# Assign Humidity as the weight.
weight=citiesna["Humidity"]
#weight.shape

In [8]:
# Plot Heatmap
fig = gmaps.figure()
# Create heat layer
heat_layer = gmaps.heatmap_layer(locations,weights=weight,dissipating=False, max_intensity=100,point_radius=3)
# Add layer
fig.add_layer(heat_layer)
# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

In [9]:
!jupyter nbextension enable --py gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

* A max temperature lower than 80 degrees but higher than 70.
* Wind speed less than 10 mph.
* Zero cloudiness.
* Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
* Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a * reasonable number.

In [10]:
#Some conditionsor a lot condition looks that my data is empty 
FittingWeather=cities_pd.loc[(cities_pd["Wind Speed"]< 10) & (cities_pd["Cloudiness"]==0)].dropna() 
#(cities_pd["Max Temp"]> 70) & (cities_pd["Max Temp"]< 80)].dropna()              

In [11]:
#Checking the Fitting Data 
FittingWeather.shape

(98, 10)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
#Store into variable named hotel_df
hotel_df=FittingWeather

In [13]:
#Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""

In [14]:
#Checking the final data with the conditions 
hotel_df.shape

(98, 11)

In [15]:
#Set parameters to search for hotels with 5000 meters.
#Hit the Google Places API for each city's coordinates. #Store in data frame hote_info
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [16]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Results for  {index}: {city}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f" The closest hotel in {city} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing result ... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    #time.sleep(1)

# Print end of search once searching is completed
print("-------End of Process-------")

Results for  14: punta arenas.
 The closest hotel in punta arenas is Hotel Cabo de Hornos.
------------
Results for  27: serowe.
 The closest hotel in serowe is Serowe Hotel (P562.00).
------------
Results for  41: saint-philippe.
 The closest hotel in saint-philippe is Les Embruns Du Baril.
------------
Results for  48: dunmore town.
 The closest hotel in dunmore town is Pink Sands Resort.
------------
Results for  50: mocuba.
 The closest hotel in mocuba is Pensão Venus.
------------
Results for  63: atar.
 The closest hotel in atar is Odar kanawal.
------------
Results for  67: arraial do cabo.
 The closest hotel in arraial do cabo is Orlanova Boutique Hotel.
------------
Results for  72: urumqi.
 The closest hotel in urumqi is Urumqi Bintuan Grand Hotel.
------------
Results for  91: beberibe.
 The closest hotel in beberibe is Coliseum Beach Resort.
------------
Results for  92: castro.
 The closest hotel in castro is Hotel Buganville Express.
------------
Results for  99: asau.
 T

Missing result ... skipping.
------------
Results for  521: brodokalmak.
Missing result ... skipping.
------------
Results for  539: san vicente.
 The closest hotel in san vicente is Hotel Y Restaurante Los Lapachos.
------------
Results for  540: mvuma.
Missing result ... skipping.
------------
Results for  542: nuuk.
 The closest hotel in nuuk is Hotel Hans Egede.
------------
Results for  545: adelaide.
 The closest hotel in adelaide is ibis Adelaide.
------------
Results for  550: concordia.
 The closest hotel in concordia is Hotel Florida.
------------
Results for  553: bhachau.
 The closest hotel in bhachau is Amrut Bagh Resort.
------------
Results for  556: tessalit.
Missing result ... skipping.
------------
Results for  557: sistranda.
 The closest hotel in sistranda is Hotell Frøya.
------------
-------End of Process-------


In [17]:
# NOTE: Do not change any of the code in this cell
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [18]:
# Add marker layer ontop of heat map
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
# Add the layer to the map
fig.add_layer(markers)
# Display Map
fig

Figure(layout=FigureLayout(height='420px'))